# Database Creation

In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine

In [5]:
with open('yelp_academic_dataset_business.json', 'r', encoding='utf-8') as f:
    business_data = [json.loads(line) for line in f]
business_df = pd.DataFrame(business_data)

with open('yelp_academic_dataset_checkin.json','r') as f:
    checkin_data = [json.loads(line) for line in f]
checkin_df = pd.DataFrame(checkin_data)


In [7]:
with open('yelp_academic_dataset_tip.json','r',encoding='utf-8') as f:
    tip_data = [json.loads(line) for line in f]
tip_df = pd.DataFrame(tip_data)


In [9]:
with open('yelp_academic_dataset_user.json','r',encoding='utf-8') as f:
    user_data = [json.loads(line) for line in f]
user_df = pd.DataFrame(user_data)

In [3]:
import pandas as pd
import dask.dataframe as dd

# Load a manageable sample using Pandas
df = pd.read_json('yelp_academic_dataset_review.json', lines=True, nrows=100000)

# Save as Parquet for efficient future access
df.to_parquet('yelp_reviews.parquet')

# Now load the Parquet file with Dask
review_df = dd.read_parquet('yelp_reviews.parquet')

# Example processing: compute the number of reviews
total_reviews = review_df.shape[0].compute()
print(f'Total number of reviews: {total_reviews}')


Total number of reviews: 100000


In [11]:
print(business_df.shape)
print(checkin_df.shape)
print(review_df.shape)
print(tip_df.shape)
print(user_df.shape)

(150346, 14)
(131930, 2)
(<dask_expr.expr.Scalar: expr=ReadParquetFSSpec(32d3e87).size() // 9, dtype=int32>, 9)
(908915, 5)
(1987897, 22)


In [13]:
business_df.drop(['attributes','hours'],axis = 1, inplace = True)

In [3]:
import dask.dataframe as dd
from sqlalchemy import create_engine

# Create a SQLAlchemy engine for SQLite
engine = create_engine('sqlite:///Yelp.db')

def load_dataframe(df, table_name, engine):
    # Convert Dask DataFrame to Pandas DataFrame
    df_pd = df.compute()  # This computes the Dask DataFrame into a Pandas DataFrame
    df_pd.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Define a function to read the JSON file in chunks
def load_json_to_dask_dataframe(file_path):
    return dd.read_json(file_path, lines=True, blocksize='64MB')  # Adjust blocksize as needed

# Load JSON files as Dask DataFrames
business_df = load_json_to_dask_dataframe('yelp_academic_dataset_business.json')
checkin_df = load_json_to_dask_dataframe('yelp_academic_dataset_checkin.json')
review_df = load_json_to_dask_dataframe('yelp_academic_dataset_review.json')
tip_df = load_json_to_dask_dataframe('yelp_academic_dataset_tip.json')
user_df = load_json_to_dask_dataframe('yelp_academic_dataset_user.json')

# Load each DataFrame into a separate table
load_dataframe(business_df, 'business', engine)
load_dataframe(checkin_df, 'checkin', engine)
load_dataframe(review_df, 'review', engine)
load_dataframe(tip_df, 'tip', engine)
load_dataframe(user_df, 'user', engine)
